In [1]:
# Variables to change
# Features
# PCA of pixel vector
# Split of training, test data
# C value of logistic regression

In [64]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# sklearn libraries for preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# sklearn libraries for models
from sklearn.linear_model import LogisticRegression

# sklearn libraries for evaluation
from sklearn import metrics

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [69]:
# Set working directory
os.chdir('C:/Users/user/Documents/Berkeley_MIDS/W207_Machine_Learning/w207_final_project')

In [70]:
# Load first npz file as an example and get files
data_1 = np.load('et_w207_project_npz_files_5000_tmp_tmp5ocrhjnn.npz')
data_1.close
data_1.files

['arr_0', 'arr_1']

In [5]:
# Show file arr_0 example
data_1['arr_0'][:5]

array(['ozbpir-9ey6js-ggdqwo', '378ykanq', 'p366md-big834-7o23k6',
       '3zruwvl2', 'q18iae-3vnh74-79npmy'], dtype='<U20')

In [6]:
# Show file arr_1 example
data_1['arr_1'][:5]

array([[-103.939   , -116.779   , -123.68    , ..., -103.939   ,
        -116.779   , -123.68    ],
       [ -46.939003,  -74.779   , -118.68    , ...,  -57.939003,
         -84.779   , -118.68    ],
       [ 151.061   ,  138.22101 ,  131.32    , ...,  151.061   ,
         138.22101 ,  131.32    ],
       [  -3.939003,  -66.779   ,  -96.68    , ...,   -3.939003,
         -66.779   ,  -96.68    ],
       [-103.939   , -116.779   , -123.68    , ..., -103.939   ,
        -116.779   , -123.68    ]], dtype=float32)

In [7]:
# Load all data files
data_2 = np.load('et_w207_project_npz_files_5000_tmp_tmp092sag67.npz')
data_2.close
print(data_2.files)

data_3 = np.load('et_w207_project_npz_files_5000_tmp_tmpf31_pn8p.npz')
data_3.close
print(data_3.files)

data_4 = np.load('et_w207_project_npz_files_5000_tmp_tmpq5b2g4n2.npz')
data_4.close
print(data_4.files)

['arr_0', 'arr_1']
['arr_0', 'arr_1']
['arr_0', 'arr_1']


In [8]:
# Get size of each file
print(data_1['arr_0'].shape)
print(data_1['arr_1'].shape)

(878,)
(878, 150528)


In [9]:
print(data_2['arr_0'].shape)
print(data_2['arr_1'].shape)

(893,)
(893, 150528)


In [10]:
print(data_3['arr_0'].shape)
print(data_3['arr_1'].shape)

(638,)
(638, 150528)


In [11]:
print(data_4['arr_0'].shape)
print(data_4['arr_1'].shape)

(1353,)
(1353, 150528)


In [12]:
# Concatenate image vectors from all files
# Concatenate data from all files
data_key = np.concatenate((data_1['arr_0'], data_2['arr_0'], data_3['arr_0'], data_4['arr_0']))
print(data_key.shape)

(3762,)


In [13]:
data_pixel = np.concatenate((data_1['arr_1'], data_2['arr_1'], data_3['arr_1'], data_4['arr_1']))
print(data_pixel.shape)

(3762, 150528)


In [14]:
# Store pixel vectors as DF with attachment key

# Create key value pairs with arr0 (attachment key) and standardized arr1 (pixel array)
# Standardize pixel values between 0 and 1 by dividing by 255
data_dict = {}
for i in range(data_key.shape[0]):
    data_dict[data_key[i]] = [data_pixel[i]/255]

df_pixel = pd.DataFrame(data=data_dict)
df_pixel_t = df_pixel.transpose()
df_pixel_t.index.names = ['attachment_key']
df_pixel_t.columns = ['pixel_array']
df_pixel_t

,pixel_array
attachment_key,
ozbpir-9ey6js-ggdqwo,"[-0.40760392, -0.45795685, -0.4850196, -0.4076..."
378ykanq,"[-0.18407452, -0.29325098, -0.46541175, -0.097..."
p366md-big834-7o23k6,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
3zruwvl2,"[-0.015447071, -0.26187843, -0.37913725, -0.01..."
q18iae-3vnh74-79npmy,"[-0.40760392, -0.45795685, -0.4850196, -0.4076..."
...,...
pktyig-481wyg-6gyulf,"[-0.3605451, -0.41089803, -0.38305882, -0.3644..."
zu7y3rha,"[-0.007603933, -0.05795686, -0.08501961, -0.00..."
pxcwxf-8tnpyg-8vhb9r,"[0.3022, 0.18125883, 0.12674509, 0.31004313, 0..."


In [15]:
# Load excel data
df = pd.read_csv('school_project_data_set-new.csv')
df.head()

,attachment_key,size_mb,height,width,h_to_w,filename,logo
0,001lq61k,0.122520,514.0,720.0,0.713889,campus ministry on the beach.jpg,0
1,002dlirq,3.256944,4072.0,3868.0,1.052741,BIG ALS FISH FLASH UV VP PL.jpg,0
2,003qefgm,0.002522,80.0,200.0,0.400000,netix_email_studio.png,1
3,0083okjl,0.116802,600.0,600.0,1.000000,productshot2.jpg,0
4,00a957mh,0.140714,816.0,2382.0,0.342569,divine_medical_billing_inc_master.png,1


In [16]:
# Merge dataframes by attachment key
merge_meta_pixel = df.merge(df_pixel_t, on='attachment_key')
merge_meta_pixel

,attachment_key,size_mb,height,width,h_to_w,filename,logo,pixel_array
0,00xjny6u,0.011849,520.0,506.0,1.027668,Esterdale Theatre - Logo.png,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
1,04o31jop,0.012610,269.0,396.0,0.679293,BLUE_LOGO.png,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
2,0840h7ox,0.399114,518.0,920.0,0.563044,ZEN_LB-5.jpg,0,"[0.106121555, 0.05576863, 0.052235294, 0.09043..."
3,0awudx2h,0.011226,200.0,200.0,1.000000,article central 200.jpg,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
4,0bb8y85h,0.263813,885.0,800.0,1.106250,1116-100_WGTank_white.jpg,0,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
...,...,...,...,...,...,...,...,...
3750,zu7y3rha,0.147371,498.0,702.0,0.709402,journalism.jpg,0,"[-0.007603933, -0.05795686, -0.08501961, -0.00..."
3751,zuanbsqe,0.580392,206.0,415.0,0.496386,2013 05 26_techstars_id_final_bug solo graysca...,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
3752,zw9234v4,2.021135,1645.0,1450.0,1.134483,SPIN N GLO SILVER WING GPK MY.jpg,0,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
3753,zxfqbfj1,0.012057,36.0,345.0,0.104348,cornerstone web logo medium.png,1,"[-0.40760392, -0.45795685, -0.4850196, -0.4076..."


In [17]:
# Remove rows with null values
merge_meta_pixel = merge_meta_pixel[merge_meta_pixel.height.notnull()]
merge_meta_pixel.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3754 entries, 0 to 3754
Data columns (total 8 columns):
attachment_key    3754 non-null object
size_mb           3754 non-null float64
height            3754 non-null float64
width             3754 non-null float64
h_to_w            3754 non-null float64
filename          3754 non-null object
logo              3754 non-null int64
pixel_array       3754 non-null object
dtypes: float64(4), int64(1), object(3)
memory usage: 264.0+ KB


In [18]:
# Find file types from filename
file_types = []
for item in merge_meta_pixel['filename']:
    file_types.append(item[-3:].lower())

print(set(file_types))

{'peg', 'pig', 'png', 'jpe', 'lue', 'hot', 'jpg', 'ack', 'age', '73e', '0mm', 'ite'}


In [19]:
# Create list of file types from filename to append to df
filetype_encoding = []
for item in merge_meta_pixel['filename']:
    if item[-3:].lower() == 'jpg':
        filetype_encoding.append('jpg')
    elif item[-3:].lower() == 'png':
        filetype_encoding.append('png')
    else:
        filetype_encoding.append('other')

In [20]:
# Create df that includes column for file type
merge_meta_pixel['filetype'] = filetype_encoding
merge_meta_pixel = merge_meta_pixel[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w', 'filetype', 'pixel_array', 'filename','logo']]
merge_meta_pixel.head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",1116-100_WGTank_white.jpg,0


In [21]:
# Get dummies for file type
filetype_dummies = pd.get_dummies(merge_meta_pixel.filetype)

# add to initial df
merge_meta_pixel = pd.concat((merge_meta_pixel, filetype_dummies), axis=1)

# print head of df
merge_meta_pixel.head()

,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo,jpg,other,png
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",Esterdale Theatre - Logo.png,1,0,0,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",BLUE_LOGO.png,1,0,0,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",ZEN_LB-5.jpg,0,1,0,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",article central 200.jpg,1,1,0,0
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",1116-100_WGTank_white.jpg,0,1,0,0


In [22]:
# Organize df to separate into feature arrays
merge_meta_pixel = merge_meta_pixel[['attachment_key', 'pixel_array', 'size_mb', 'h_to_w', 'jpg', 'png', 'other',
                                     'height', 'width', 'logo', 'filetype', 'filename']]
merge_meta_pixel.head()

,attachment_key,pixel_array,size_mb,h_to_w,jpg,png,other,height,width,logo,filetype,filename
0,00xjny6u,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",0.011849,1.027668,0,1,0,520.0,506.0,1,png,Esterdale Theatre - Logo.png
1,04o31jop,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",0.012610,0.679293,0,1,0,269.0,396.0,1,png,BLUE_LOGO.png
2,0840h7ox,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",0.399114,0.563044,1,0,0,518.0,920.0,0,jpg,ZEN_LB-5.jpg
3,0awudx2h,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",0.011226,1.000000,1,0,0,200.0,200.0,1,jpg,article central 200.jpg
4,0bb8y85h,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",0.263813,1.106250,1,0,0,885.0,800.0,0,jpg,1116-100_WGTank_white.jpg


In [23]:
# Scale continuous feature variables
merge_meta_pixel['size_mb'] = preprocessing.scale(merge_meta_pixel['size_mb'])
merge_meta_pixel['h_to_w'] = preprocessing.scale(merge_meta_pixel['h_to_w'])
merge_meta_pixel['height'] = preprocessing.scale(merge_meta_pixel['height'])
merge_meta_pixel['width'] = preprocessing.scale(merge_meta_pixel['width'])

# Standardize data
#scaler = StandardScaler() 
#scaled_df = scaler.fit_transform(raw_df) 
  
# Normalizing the Data 
#normalized_df = normalize(scaled_df) 
  
# Converting the numpy array into a pandas DataFrame 
#normalized_df = pd.DataFrame(normalized_df) 

In [24]:
# y array contains logo/not logo binary variable
y = np.array(merge_meta_pixel.iloc[:, 9])

# X array contains pixel array only
X = np.array(merge_meta_pixel['pixel_array'].tolist())
print(X.shape)

# Reducing dimensions of the pixel data
def reduce(X, n):
    '''PCA dimensionality reduction for X array and n principal components'''
    pca = PCA(n_components = n) 
    X_reduced = pca.fit_transform(X)
    return X_reduced

X_reduced = reduce(X,2)

# Create variations of X array with reduced pixel data and different metadata features

# X2 array contains pixel features, size, height, width, h_to_w, filetype dummies
X2 = np.concatenate((X_reduced, np.array(merge_meta_pixel.iloc[:, 2:9])),axis=1)
print(X2.shape)

# X3 array contains pixel features, size, h_to_w, filetype dummies
X3 = np.concatenate((X_reduced, np.array(merge_meta_pixel.iloc[:, 2:7])),axis=1)
print(X3.shape)

# X4 array contains pixel array, size, h_to_w
X4 = np.concatenate((X_reduced, np.array(merge_meta_pixel.iloc[:, 2:4])),axis=1)
print(X4.shape)

# X5 array contains pixel array, filetype dummies 
X5 = np.concatenate((X_reduced, np.array(merge_meta_pixel.iloc[:, 4:7])),axis=1)
print(X5.shape)

# X6 array contains size, height, width, h_to_w, filetype
X6 = np.array(merge_meta_pixel.iloc[:, 2:9])
print(X6.shape)

# X7 array contains size, h_to_w, filetype
X7 = np.array(merge_meta_pixel.iloc[:, 2:7])
print(X7.shape)

(3754, 150528)
(3754, 9)
(3754, 7)
(3754, 4)
(3754, 5)
(3754, 7)
(3754, 5)


In [25]:
def data_split(X, y, test_p, mini_s):
    '''Split train and test data for X,y data arrays with test_p percentage of observations in the test data and 
    mini_s number of observations in the mini_train set'''
    # Split into train and test data
    train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=test_p, random_state=1, stratify=y)

    # Split test data into development data and training into mini train set for computation time
    dev_data, dev_labels = test_data[:int(test_data.shape[0]*0.5)], test_labels[:int(test_data.shape[0]*0.5)]
    mini_train_data, mini_train_labels = train_data[:mini_s], train_labels[:mini_s]
    print('Train data shape:', train_data.shape)
    print('Dev data shape:', dev_data.shape)
    print('Mini_train data shape:', mini_train_data.shape)
    
    return train_data, train_labels, test_data, test_labels, dev_data, dev_labels, mini_train_data, mini_train_labels

In [26]:
# Split all feature combinations into train, test data
train_data, train_labels, test_data, test_labels, dev_data, dev_labels, mini_train_data, mini_train_labels = data_split(X_reduced, y, 0.4, 100)
train_data2, train_labels2, test_data2, test_labels2, dev_data2, dev_labels2, mini_train_data2, mini_train_labels2 = data_split(X2, y, 0.4, 100)
train_data3, train_labels3, test_data3, test_labels3, dev_data3, dev_labels3, mini_train_data3, mini_train_labels3 = data_split(X3, y, 0.4, 100)
train_data4, train_labels4, test_data4, test_labels4, dev_data4, dev_labels4, mini_train_data4, mini_train_labels4 = data_split(X4, y, 0.4, 100)
train_data5, train_labels5, test_data5, test_labels5, dev_data5, dev_labels5, mini_train_data5, mini_train_labels5 = data_split(X5, y, 0.4, 100)
train_data6, train_labels6, test_data6, test_labels6, dev_data6, dev_labels6, mini_train_data6, mini_train_labels6 = data_split(X6, y, 0.4, 100)
train_data7, train_labels7, test_data7, test_labels7, dev_data7, dev_labels7, mini_train_data7, mini_train_labels7 = data_split(X7, y, 0.4, 100)

Train data shape: (2252, 2)
Dev data shape: (751, 2)
Mini_train data shape: (100, 2)
Train data shape: (2252, 9)
Dev data shape: (751, 9)
Mini_train data shape: (100, 9)
Train data shape: (2252, 7)
Dev data shape: (751, 7)
Mini_train data shape: (100, 7)
Train data shape: (2252, 4)
Dev data shape: (751, 4)
Mini_train data shape: (100, 4)
Train data shape: (2252, 5)
Dev data shape: (751, 5)
Mini_train data shape: (100, 5)
Train data shape: (2252, 7)
Dev data shape: (751, 7)
Mini_train data shape: (100, 7)
Train data shape: (2252, 5)
Dev data shape: (751, 5)
Mini_train data shape: (100, 5)


In [27]:
# Standardize pixel values between 0 and 1 by dividing by 255
#data = [train_data, test_data, dev_data, mini_train_data]

#print(train_data)

#for element in data:
#    for array in element:
#        for value in array:
#            value = value/255

**Logistic Regression**

In [28]:
def log_reg(train_X, train_Y, dev_X, dev_Y):
    '''Logistic regression looping through values of C'''
    c_values = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 5.0]
    LR = []
    for c in c_values:
        clf = LogisticRegression(C=c, solver='liblinear', multi_class='auto')
        clf.fit(train_X, train_Y)
        clf_pred = clf.predict(dev_X)
        LR.append(round(metrics.f1_score(dev_Y, clf_pred, average='weighted'),3))

    return LR, c_values

In [29]:
# FOR REFERENCE
# X array contains pixel array only
# X2 array contains pixel array, size, h_to_w, filetype dummies
# X3 array contains pixel array, size, h_to_w
# X4 array contains pixel array, size
# X5 array contains pixel array, h_to_w 
# X6 array contains pixel array, filetype dummies 

In [30]:
def LR_accuracy_c(train_X, train_Y, dev_X, dev_Y):
    accuracy_list = log_reg(train_X, train_Y, dev_X, dev_Y)[0]
    accuracy = [y for x,y in sorted(enumerate(accuracy_list), key = lambda x: x[1], reverse=True)][:1]
    accuracy_index = [x for x,y in sorted(enumerate(accuracy_list), key = lambda x: x[1], reverse=True)][:1]
    C = log_reg(train_X, train_Y, dev_X, dev_Y)[1][accuracy_index[0]]
    
    return accuracy[0], C

In [31]:
# Pixel array only
# Run LR and output best accuracy with c value
print('Features: pixel features only\nBest Accuracy: {}, c={}'.format(
    LR_accuracy_c(train_data, train_labels, dev_data, dev_labels)[0],
    LR_accuracy_c(train_data, train_labels, dev_data, dev_labels)[1]))

Features: pixel features only
Best Accuracy: 0.577, c=0.001


In [32]:
# Run LR and output best accuracies with c values for each combination of features
print('Features: pixel features, size, height, width, h_to_w, filetype dummies\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data2, train_labels2, dev_data2, dev_labels2)[0],
    LR_accuracy_c(train_data2, train_labels2, dev_data2, dev_labels2)[1]))

print('Features: pixel features, size, h_to_w, filetype dummies\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data3, train_labels3, dev_data3, dev_labels3)[0],
    LR_accuracy_c(train_data3, train_labels3, dev_data3, dev_labels3)[1]))

print('Features: pixel features, size, h_to_w\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data4, train_labels4, dev_data4, dev_labels4)[0],
    LR_accuracy_c(train_data4, train_labels4, dev_data4, dev_labels4)[1]))

print('Features: pixel features, filetype dummies\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data5, train_labels5, dev_data5, dev_labels5)[0],
    LR_accuracy_c(train_data5, train_labels5, dev_data5, dev_labels5)[1]))

print('Features: size, height, width, h_to_w, filetype\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data6, train_labels6, dev_data6, dev_labels6)[0],
    LR_accuracy_c(train_data6, train_labels6, dev_data6, dev_labels6)[1]))

print('Features: size, h_to_w, filetype\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data7, train_labels7, dev_data7, dev_labels7)[0],
    LR_accuracy_c(train_data7, train_labels7, dev_data7, dev_labels7)[1]))

Features: pixel features, size, height, width, h_to_w, filetype dummies
Best Accuracy: 0.858, c=0.1

Features: pixel features, size, h_to_w, filetype dummies
Best Accuracy: 0.849, c=2.0

Features: pixel features, size, h_to_w
Best Accuracy: 0.768, c=0.3

Features: pixel features, filetype dummies
Best Accuracy: 0.822, c=0.2

Features: size, height, width, h_to_w, filetype
Best Accuracy: 0.845, c=0.1

Features: size, h_to_w, filetype
Best Accuracy: 0.849, c=0.01



In [33]:
log_reg(train_data2, train_labels2, dev_data2, dev_labels2)

([0.803, 0.841, 0.858, 0.853, 0.849, 0.849, 0.848, 0.848, 0.85, 0.85],
 [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 5.0])

**Multi-Layer Neural Network**  
**Tensorflow**

In [66]:
# Function to build layers of neural net

def build_model(hp):
    # Begins the linear arrangment of layers
    model = tf.keras.models.Sequential([

    # Creates a fully-connected layer. A layer is fully connected to the layer that preceds it.
    # Dense1_units number of nodes in the layer, choses a value between 1 and 100 based on optimization 
      tf.keras.layers.Dense(units=hp.Choice('dense1_units', values=[1, 100]), activation='relu'),

    # Uses dropout regularization by dropping 20% before next layer
      tf.keras.layers.Dropout(0.2),

    # 2 nodes in final output for logo/not logo
      tf.keras.layers.Dense(2, activation='softmax') 
    ])
    
    model.compile(optimizer=tf.keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

# Other activation functions: activation='sigmoid'
# Other optimizers to try - Adam

In [89]:
file_path = 'C:\\Users\\user\\Documents\\Berkeley_MIDS\\W207_Machine_Learning\\w207_final_project'

In [92]:
# Use keras random search with build_model function to maximize accuracy 
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='C:\\Users\\user\\Documents\\Berkeley_MIDS\\W207_Machine_Learning\\w207_final_project',
    project_name='keras_tuner')

INFO:tensorflow:Reloading Oracle from existing project C:\Users\user\Documents\Berkeley_MIDS\W207_Machine_Learning\w207_final_project\keras_tuner\oracle.json


In [93]:
# Start search with training data
tuner.search(train_data2, train_labels2,
             epochs=10,
             validation_data=(dev_data2, dev_labels2))

Epoch 1/10
55/71 [======================>.......] - ETA: 0s - loss: 3.3347 - accuracy: 0.59 - ETA: 0s - loss: 3.4129 - accuracy: 0.5920

NotFoundError: Failed to create a NewWriteableFile: C:\Users\user\Documents\Berkeley_MIDS\W207_Machine_Learning\w207_final_project\keras_tuner\trial_99834874d955a686d083597d5168c5ee\checkpoints\epoch_0\checkpoint_temp_df2054dae9294905a733c5789e930905/part-00000-of-00001.data-00000-of-00001.tempstate15128673017566932951 : The system cannot find the path specified.
; No such process [Op:SaveV2]

In [72]:
# Get results
tuner.get_best_models(num_models=2)

[]


In [ ]:
#this specifies what kind of layers are going to be present in the neural net
#Begins the linear arrangment of layers
model = tf.keras.models.Sequential([
    
  #tf.keras.layers.Flatten(input_shape=(150528, 1)),
    
#Creates a fully-connected layer. A layer is fully connected to the layer that preceds it.
#128 nodes in the layer
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='softmax') 
])

In [58]:
#Epoch = number of times the model sees all of the training
model.fit(train_data2, train_labels2, epochs=10)

Epoch 1/10
71/71 [==============================] - 0s 871us/step - loss: 0.4707 - accuracy: 0.7882
Epoch 2/10
71/71 [==============================] - 0s 786us/step - loss: 0.4720 - accuracy: 0.7855
Epoch 3/10
71/71 [==============================] - 0s 815us/step - loss: 0.4696 - accuracy: 0.7829
Epoch 4/10
71/71 [==============================] - 0s 773us/step - loss: 0.4724 - accuracy: 0.7886
Epoch 5/10
71/71 [==============================] - 0s 801us/step - loss: 0.4714 - accuracy: 0.7771
Epoch 6/10
71/71 [==============================] - 0s 942us/step - loss: 0.4743 - accuracy: 0.7740
Epoch 7/10
71/71 [==============================] - 0s 829us/step - loss: 0.4664 - accuracy: 0.7909
Epoch 8/10
71/71 [==============================] - 0s 815us/step - loss: 0.4702 - accuracy: 0.7815
Epoch 9/10
71/71 [==============================] - 0s 829us/step - loss: 0.4659 - accuracy: 0.7842
Epoch 10/10
71/71 [==============================] - 0s 759us/step - loss: 0.4692 - accuracy: 0.7749

In [37]:
train_data2.shape

(2252, 9)

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                500       
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102       
Total params: 602
Trainable params: 602
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.evaluate(dev_data2, dev_labels2)

24/24 [==============================] - 0s 582us/step - loss: 0.5121 - accuracy: 0.7483


[0.5120877623558044, 0.7483355402946472]